# trace

In [ ]:
//// test

open testing

()



## types

In [ ]:
inl types () =
    rust.types ()
    sm'.types ()
    env.types ()

()



## trace_level

In [ ]:
union trace_level =
    | Verbose
    | Debug
    | Info
    | Warning
    | Critical

instance to_string trace_level = function
    | Verbose => "Verbose"
    | Debug => "Debug"
    | Info => "Info"
    | Warning => "Warning"
    | Critical => "Critical"

instance of_string trace_level = function
    | "Verbose" => Verbose
    | "Debug" => Debug
    | "Info" => Info
    | "Warning" => Warning
    | "Critical" => Critical
    | x => failwith $'$"of_string trace_level / x: {!x}"'

()



## repl_start

In [ ]:
inl repl_start () : optionm'.option' i64 =
    run_target function
        | Fsharp (Native) & x
        | Rust (Native) & x => fun () =>
            inl automation = env.get_environment_variable "AUTOMATION"
            // inl target =
            //     match x with
            //     | Fsharp (Native) => "Fsharp (Native)"
            //     | Rust (Native) => "Rust (Native)"
            // inl assembly_name = env.get_entry_assembly_name ()
            // console.write_line ($'$"repl_start / automation: \'{!automation}\' / target: {!target} / assembly_name: {!assembly_name}"' : string)
            if automation = "True"
            then date_time.now () |> date_time.ticks |> fun (date_time.timestamp x) => x |> Some
            else None
            |> optionm'.box
        | _ => fun () => optionm'.none' ()

()



## trace_state

In [ ]:
type trace_state =
    {
        count : mut i64
        trace_file : mut (string -> ())
        enabled : mut bool
        level : mut trace_level
        repl_start : optionm'.option' i64
    }

inl new_trace_state trace_level =
    {
        enabled = mut true
        count = mut 0i64
        level = mut trace_level
        trace_file = mut ignore
        repl_start = repl_start ()
    } : trace_state

()



## init_trace_state

In [ ]:
inl init_trace_state trace_level : () =
    inl trace_level =
        trace_level
        |> optionm'.default_value Verbose
    global "module State = let mutable trace_state = None"
    $"if State.trace_state.IsNone then State.trace_state <- !new_trace_state !trace_level |> Some" : ()

()



## get_trace_state_or_init

In [ ]:
inl get_trace_state_or_init trace_level : trace_state =
    init_trace_state trace_level
    $'State.trace_state.Value'

()



## _locals

In [ ]:
inl _locals () = ""

()



## test_trace_level

In [ ]:
inl test_trace_level level : bool =
    inl state = get_trace_state_or_init None
    inl level' = *state.level
    if *state.enabled |> not
    then false
    else
        inl level : i32 = real real_core.union_tag level
        inl level' : i32 = real real_core.union_tag level'
        level >= level'

()



In [ ]:
//// test

test_trace_level Critical |> _assert_eq true
test_trace_level Verbose |> _assert_eq true

inl level = get_trace_state_or_init None .level
level <- Debug
test_trace_level Verbose |> _assert_eq false
level <- Verbose
test_trace_level Verbose |> _assert_eq true

module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and Mut3 = {mutable l0 : (string -> unit)}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
let rec closure1 () (v0 : string) : unit =
    ()
and method1 () : string =
    let v0 : string = ""
    v0
and method2 () : string =
    let v0 : string = "AUTOMATION"
    v0
and closure0 () (v0 : US0) : struct (Mut0 * Mut1 * Mut2 * int64 option * Mut3) =
    let v1 : Mut1 = {l0 = true} : Mut1
    let v2 : Mut0 = {l0 = 0L} : Mut0
    let v3 : Mut2 = {l0 = v0} : Mut2
    let v4 : (string -> unit) = closure1()
    let v5 : Mut3 = {l0 = v4} : Mut3
    let v6 : bool = true
    let mutable _v6 : int64 option option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v7 : bool = true
    let mutable _v7 : string o

## trace_raw

In [ ]:
let rec trace_raw level fn =
    inl trace_state = get_trace_state_or_init None
    if level |> test_trace_level then
        inl count = trace_state.count
        count <- *trace_state.count + 1

        inl text = $'$"%s{!fn ()}"' : string
        run_target function
        | Rust _ => fun () =>
            open rust_operators
            !\\(text, $'\@"println\!(""{}"", $0)"')
        | _ => fun () =>
            $'System.Console.WriteLine !text '

        *trace_state.trace_file text

()



## trace

In [ ]:
let trace (level : trace_level) (text_fn : () -> string) (locals : () -> string) =
    fun () =>
        inl trace_state = get_trace_state_or_init None
        inl time =
            run_target fun target =>
                match target with
                | Fsharp (Native)
                | Rust (Native) => fun () =>
                    match trace_state.repl_start |> optionm'.unbox with
                    | Some repl_start =>
                        inl t =
                            (date_time.now () |> date_time.ticks |> fun (date_time.timestamp x) => x)
                            - repl_start |> date_time.time_span
                        date_time.date_time_milliseconds
                            1i32 1i32 1i32
                            (t |> date_time.hours)
                            (t |> date_time.minutes)
                            (t |> date_time.seconds)
                            (t |> date_time.milliseconds)
                    | None => date_time.now ()
                    |> fun date_time =>
                        inl format =
                            match target with
                            | Rust _ => "hh:mm:ss"
                            | _ => "HH:mm:ss"
                        $'!date_time.ToString !format '
                | _ => fun () => join ""
        inl count = *trace_state.count
        inl level_str = level |> to_string |> sm'.to_lower |> sm'.pad_left 7 ' '
        inl level_str =
            run_target function
            | Rust _ => fun () =>
                open rust_operators
                inl color : rust.ref' sm'.str =
                    match level with
                    | Verbose => !\($'"inline_colorization::color_bright_black"')
                    | Debug => !\($'"inline_colorization::color_bright_blue"')
                    | Info => !\($'"inline_colorization::color_bright_green"')
                    | Warning => !\($'"inline_colorization::color_yellow"')
                    | Critical => !\($'"inline_colorization::color_bright_red"')
                inl level_str = level_str |> sm'.as_str
                inl color_reset : rust.ref' sm'.str = !\($'"inline_colorization::color_reset"')
                $'"\\\"{!color}{!level_str}{!color_reset}\\\""'
                |> sm'.format''
                |> sm'.from_std_string
            | _ => fun () => level_str
        $'$"{!time} {!level_str} #{!count} %s{!text_fn ()} / %s{!locals ()}"'
        |> sm'.trim_start ;[]
        |> sm'.trim_end ;[ ' '; '/' ]
    |> trace_raw level

()



In [ ]:
//// test
//// print_code=false

trace Debug (fun () => "test") _locals
trace Debug (fun () => "test") _locals
get_trace_state_or_init None .count
|> fun x => *x
|> _assert_eq 2

00:00:00   debug #1 test
00:00:00   debug #2 test
assert_eq / actual: 2L / expected: 2L


In [ ]:
//// test
//// print_code=false
///! rust

types ()
trace Debug (fun () => "test1") _locals
trace Debug (fun () => "test2") _locals
get_trace_state_or_init None .count
|> fun x => *x
|> _assert_eq 2

00:00:00   debug #1 test1
00:00:00   debug #2 test2
assert_eq / actual: 2 / expected: 2


## main

In [ ]:
inl main () =
    types ()
    init_trace_state None
    $"let trace x = !trace x" : ()

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("_")>]
#endif
type Any = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type Actio